# Example of using `cells_in_gel`

## Imports

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob

from skimage.morphology import disk

from cells_in_gel import preprocess as pp
from cells_in_gel import properties as props
from cells_in_gel import batch

## Load Images

Load images from a filepath using `glob.glob('filenames.tif')` into a list of strings. 

In [ ]:
# list of filenames
files = glob.glob('C3*.tif') 

Using the `cells_in_gel.batch.max_projection(files)`, input the list of filenames to create a dictionary of max intensity projections. 

In [ ]:
max_proj = batch.max_projection(files)

Here is a plot showing the max intensity projection from the 19th filename in the list of `files`.

In [ ]:
plt.imshow(max_proj[files[18]])

## Segmentation and Region Properties

Create the `selem` for separation of objects. A higher number will result in less separation between objects in max intensity projections. (Need to update our original preprocess function to optimize this a bit better).

In [ ]:
selem = disk(1)

Using `cells_in_gel.batch.labels_regions(files)`, segment max projections, label objects, and extract region properties. The output `regions` will give a dictionary containing dataframes of properties for each max intensity projection. 

In [ ]:
regions = batch.labels_regions(files, selem, min_size=350)

Here is the head for the first max intensity projection dataframe in our list of `files`.

In [ ]:
regions[files[0]].head()

## Extracting Average Values from Max Intesnity Projections

Create a list of `titles` that describe categories between each underscore in filenames. Create a list of average `properties` to extract from the max intensity projection regions dataframe.

In [ ]:
titles = ['celltype', 'ecmtype_conc', 'rgd_conc', 'mag', 'img_num']
properties = ['area', 'eccentricity', 'extent']

Using `cells_in_gel.batch.image_summaries(files, properties, titles, regions)`, extract a summary dataframe that includes average and variance properties from each max intensity projection. See the `summary` head below. 

In [ ]:
summary = batch.image_summaries(files, properties, titles, regions)
summary.head()

Further analysis of grouping, etc. will be done using the `summary` dataframe. At this point, you can use `summary.to_csv('imagesummaries_green.csv', index=False)` for creating a CSV of this dataframe.

## WORK IN PROGRESS: Graphing Groups

Playing around with `groupby`, but not sure how to extract values from the dataframe grouped in this way. 

In [ ]:
groups = ['celltype', 'ecmtype_conc', 'rgd_conc']

summary_g = summary.groupby(groups).mean()
summary_g

Instead, tried grouping in a different way, and this seems easier to extract information from.

In [ ]:
groups = ['celltype', 'ecmtype_conc', 'rgd_conc']
ecms = ['I61Q1ECM', 'I61Q5ECM', 'NTG1ECM', 'NTG5ECM', 'NoECM']
rgd = ['0mMRGD', '1mMRGD']

gr = {}
gr_rgd = {}

i = 0
for ecm in ecms:
    gr[i] = summary.groupby(groups).get_group(('C3-NTG-CFbs', ecm, rgd[0]))
    i += 1

i = 0
for ecm in ecms:
    gr_rgd[i] = summary.groupby(groups).get_group(('C3-NTG-CFbs', ecm, rgd[1]))
    i += 1
    
gr[0]

**Here is my super hackey way of getting to the graphs although I am definitely planning to clean this up! It's probably very confusing at this point...**

In [ ]:
summary[3:6]

In [ ]:
g1_no_rgd = np.mean(summary[0:3]['area'])
g1_rgd = np.mean(summary[4:6]['area'])

g2_no_rgd = np.mean(summary[6:9]['area']) 
g2_rgd = np.mean(summary[9:12]['area'])

g3_no_rgd = np.mean(summary[12:15]['area']) 
g3_rgd = np.mean(summary[15:18]['area']) 

g4_no_rgd = np.mean(summary[18:21]['area']) 
g4_rgd = np.mean(summary[21:24]['area']) 

g5_no_rgd = np.mean(summary[24:27]['area']) 
g5_rgd = np.mean(summary[27:]['area']) 

In [ ]:
g1_no_rgd_e = np.mean(summary[0:3]['eccentricity'])
g1_rgd_e = np.mean(summary[4:6]['eccentricity'])

g2_no_rgd_e = np.mean(summary[6:9]['eccentricity']) 
g2_rgd_e = np.mean(summary[9:12]['eccentricity'])

g3_no_rgd_e = np.mean(summary[12:15]['eccentricity']) 
g3_rgd_e = np.mean(summary[15:18]['eccentricity']) 

g4_no_rgd_e = np.mean(summary[18:21]['eccentricity']) 
g4_rgd_e = np.mean(summary[21:24]['eccentricity']) 

g5_no_rgd_e = np.mean(summary[24:27]['eccentricity']) 
g5_rgd_e = np.mean(summary[27:]['eccentricity']) 

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np


labels = ['I61Q 1mg/mL', 'I6IQ 5mg/mL', 'No ECM', 'NTG 1mg/mL', 'NTG 5mg/mL']
g1_areas = [g1_no_rgd, g2_no_rgd, g3_no_rgd, g4_no_rgd, g5_no_rgd]
g2_areas = [g1_rgd, g2_rgd, g3_rgd, g4_rgd, g5_rgd]
g1_ecc = [g1_no_rgd_e, g2_no_rgd_e, g3_no_rgd_e, g4_no_rgd_e, g5_no_rgd_e]
g2_ecc = [g1_rgd_e, g2_rgd_e, g3_rgd_e, g4_rgd_e, g5_rgd_e]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(ncols=2, figsize=(14,5))

ax[0].bar(x - width/2, g1_areas, width, label='No RGD')
ax[0].bar(x + width/2, g2_areas, width, label='RGD')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax[0].set_ylabel('Mean Cell Area (pixels)')
ax[0].set_title('NTG Fibroblasts')
ax[0].set_xticks(x)
ax[0].set_xticklabels(labels)
ax[0].legend()

ax[1].bar(x - width/2, g1_ecc, width, label='No RGD')
ax[1].bar(x + width/2, g2_ecc, width, label='RGD')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax[1].set_ylabel('Mean Eccentricity')
ax[1].set_title('NTG Fibroblasts')
ax[1].set_xticks(x)
ax[1].set_xticklabels(labels)
ax[1].legend()


fig.tight_layout()

plt.show()